# BERT 임베딩  
[BERT Word Embeddings Tutorial](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/)  
[BERT Word Embeddings 튜토리얼 번역](https://codlingual.tistory.com/m/98)  

In [ ]:
!  pip install pytorch-pretrained-bert

In [19]:
# Pre-trained BERT 불러오기
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [20]:
# Tokenization
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS]" + text + "[SEP]"

tokenized_text = tokenizer.tokenize(marked_text)
print(tokenized_text)

['[', 'cl', '##s', ']', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[', 'sep', ']']


In [21]:
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."
marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
after         2,044
stealing     11,065
money         2,769
from          2,013
the           1,996
bank          2,924
vault        11,632
,             1,010
the           1,996
bank          2,924
robber       27,307
was           2,001
seen          2,464
fishing       5,645
on            2,006
the           1,996
mississippi   5,900
river         2,314
bank          2,924
.             1,012
[SEP]           102


In [22]:
# Segment ID

segments_ids = [1] * len(tokenized_text)

In [23]:
# - 인풋 문장이 하나면 segment ID 모두 1로 주면 됨 (인풋 문장 + 마지막 [SEP] 까지 1 주기)
# - 인풋 문장이 두개면 첫 문장은 0, 다음 문장은 1로 줘서 구분하기 

# Extracting Embeddings

# Python list를 PyTorch tensor로 변환하기 
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# 미리 학습된 모델(가중치) 불러오기
model = BertModel.from_pretrained('bert-base-uncased')

# 모델 "evaluation" 모드 : feed-forward operation
model.eval()

# 우린 forward pass만 하지 [오차역전파해서 가중치 업데이트, 학습시키기] 이런거 안 하니까 no_grad()
# no_grad() 쓰면 계산 더 빠름 

# 각 레이어의 은닉상태 확인하기
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
 

* Encoded_layers

    1) layer 개수 : len(encoded_layers) # 12개

    2) batch 개수 : len(encoded_layers[0]) # 1개

    3) 단어/토큰 개수 :  len(encoded_layers[0][0]) # 22개

    4) 은닉 상태 차원(hidden units/features) : len(encoded_layers[0][0][0]) # 768개


* 현재 encoded_layers의 차원 

    [layer 개수, batch 개수, token 개수, feature 개수]

* 바꾸고 싶은 차원 

    [token 개수, layer 개수, feature 개수]


- encoded_layers 자체는 Python list 

- 각 layer 안은 torch tensor로 이루어짐 

- encoded_layers[0].size()  # torch.Size([1,22,768])


In [24]:
# 12개의 layer를 합쳐서 하나의 큰 tensor로 만들기 
token_embeddings = torch.stack(encoded_layers, dim=0)
token_embeddings.size() # torch.Size([12, 1, 22, 768])

torch.Size([12, 1, 22, 768])

In [25]:
# batch 차원 없애기
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings.size()  # torch.Size([12, 22, 768])

torch.Size([12, 22, 768])

In [26]:
# 지금까지 하면 token_embeddings 차원은 
# [layer 개수, token 개수, feature 개수]
# 여기서 layer 개수와 token 개수의 자리만 바꾸면 됨 

token_embeddings = token_embeddings.permute(1,0,2)
token_embeddings.size()  # torch.Size([22, 12, 768])

torch.Size([22, 12, 768])

## 은닉상태로부터 단어/문장 벡터 만들기

### 1. 단어(토큰) 벡터 만들기


1) 맨 마지막 4개 레이어 이어붙이기(concatenate)

- 각 벡터의 길이는 4*768 = 3072 

- 4는 레이어 개수, 768은 feature 개수 

- 각 토큰을 3072 길이의 벡터로 나타냈는데, 총 22개의 토큰이 있음



In [27]:
token_vecs_cat = []

# token_embeddings : [22,12,768]
# token : [12,768]
for token in token_embeddings :
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    token_vecs_cat.append(cat_vec)
    
print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))
# Shape is: 22 x 3072


Shape is: 22 x 3072


2) 맨 마지막 4개 레이어 합치기(sum)

-  합치기만 했으니 한 토큰을 나타내는 벡터 길이는 여전히 768

- 토큰이 총 22개 있으니 최종 shape는 22*768

In [28]:
token_vecs_sum = []

for token in token_embeddings:
    
    sum_vec = torch.sum(token[-4:], dim=0)
    
    token_vecs_sum.append(sum_vec)
    
print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 22 x 768


## 2. 문장 벡터 만들기

- 마지막 레이어에서 모든 토큰의 은닉상태 평균 구하기 

- 평균 구했으니 벡터 길이는 여전히 768

In [29]:
# encoded_layers : [12*1*22*768]
# token_vecs : [22*768]
token_vecs = encoded_layers[11][0]

# sentence_embedding : [768]
sentence_embedding = torch.mean(token_vecs, dim=0)

In [30]:
# 동음이의어 벡터 확인하기 
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 after
2 stealing
3 money
4 from
5 the
6 bank
7 vault
8 ,
9 the
10 bank
11 robber
12 was
13 seen
14 fishing
15 on
16 the
17 mississippi
18 river
19 bank
20 .
21 [SEP]


In [31]:
print('First 5 vector values for each instance of "bank".')
print('')
print("bank vault   ", str(token_vecs_sum[6][:5]))
print("bank robber  ", str(token_vecs_sum[10][:5]))
print("river bank   ", str(token_vecs_sum[19][:5]))

First 5 vector values for each instance of "bank".

bank vault    tensor([ 2.1319, -2.1413, -1.6260,  0.8638,  3.3173])
bank robber   tensor([ 1.1868, -1.5298, -1.3770,  1.0648,  3.1446])
river bank    tensor([ 1.1295, -1.4725, -0.7296, -0.0901,  2.4970])


In [32]:
# 각 임베딩의 유사성 계산하기 

from scipy.spatial.distance import cosine

# 다른 의미의 bank 임베딩 비교 
# "bank robber" vs "river bank" (different meanings)
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# 같은 의미의 bank 임베딩 비교
# "bank robber" vs "bank vault" (same meaning)
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for  *similar*  meanings:  0.95
Vector similarity for *different* meanings:  0.68
